In [1]:
import numpy as np
import os
import cv2
import keras
import matplotlib.pyplot as plt
from imutils import paths
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [64]:
from sklearn.svm import SVC

# converting images into arrays

In [2]:
classes = ['c0', 'c1','c2','c3','c4','c5','c6','c7','c8','c9']
Dict = {'c0' : 0, 'c1' :1, 'c2':2, 'c3':3, 'c4':4,'c5':5,'c6':6,'c7':7,'c8':8,'c9':9}
images = []
Img_labels = []
train_path='imgs/train'

In [3]:
for label in classes:
      path = os.path.join(train_path , label)
      for img in os.listdir(path):
         img = cv2.imread(os.path.join(path,img))
         new_img = cv2.resize(img, (64, 64))
         images.append(new_img)
         Img_labels.append(Dict[label])
         #print(images,Img_labels)
     #print(label)

In [32]:
img=np.array(images)
labels=np.array(Img_labels)
print(img.shape)

(22424, 64, 64, 3)


In [33]:
print(img)

[[[[ 40  39  30]
   [ 22  26  21]
   [137 166 146]
   ...
   [183 168  87]
   [229 218 131]
   [254 250 176]]

  [[ 13   9   1]
   [  9  17  10]
   [ 11  10  14]
   ...
   [239 232 160]
   [254 255 170]
   [223 220 128]]

  [[  6   6   4]
   [ 87 118 101]
   [ 72 100  76]
   ...
   [241 240 146]
   [221 213 118]
   [212 205 114]]

  ...

  [[ 41  48  33]
   [ 53  57  46]
   [ 59  64  49]
   ...
   [ 14  13  15]
   [ 11  10  12]
   [ 13  12  14]]

  [[ 48  54  43]
   [ 58  60  45]
   [ 59  65  48]
   ...
   [ 13  15  16]
   [ 11  13  14]
   [ 16  18  19]]

  [[ 50  59  40]
   [ 63  64  51]
   [ 56  62  45]
   ...
   [ 13  15  16]
   [ 11  13  14]
   [ 12  14  15]]]


 [[[ 36  43  36]
   [ 35  42  35]
   [ 40  46  41]
   ...
   [221 253 212]
   [235 253 242]
   [254 254 254]]

  [[ 39  46  39]
   [ 39  46  39]
   [ 38  44  39]
   ...
   [219 236 185]
   [215 243 194]
   [233 255 230]]

  [[ 39  47  37]
   [ 41  49  39]
   [ 33  43  37]
   ...
   [225 233 183]
   [225 240 189]
   [222 242

In [44]:
labels

array([0, 0, 0, ..., 9, 9, 9])

In [45]:
full_x = img
full_y = labels

# KAZE buiding

In [ ]:
import cv2

In [ ]:
def get_kaze(images, name='kaze'):
    def get_image_kaze(image, vector_size=32):
        alg = cv2.KAZE_create()
        kps = alg.detect(image)
        kps = sorted(kps, key=lambda x: -x.response)[:vector_size]
        
        # Making descriptor of same size
        # Descriptor vector size is 64
        needed_size = (vector_size * 64)
        if len(kps) == 0:
            return np.zeros(needed_size)
        
        kps, dsc = alg.compute(image, kps)
        dsc = dsc.flatten()
        
        if dsc.size < needed_size:
            # if we have less than 32 descriptors then just adding zeros at the
            # end of our feature vector
            dsc = np.concatenate([dsc, np.zeros(needed_size - dsc.size)])
        return dsc
    
    # KAZE descriptor for all images
    features = []
    for i, img in enumerate(images):
        dsc = get_image_kaze(img)
        features.append(dsc)
    
    result = np.array(features)

# Train test and validation split

In [48]:
from sklearn.model_selection import train_test_split

In [49]:
X , X_test , y ,y_test = train_test_split(full_x , full_y , test_size=0.2, random_state=0)

In [69]:
train_imgs , val_imgs , train_y , val_y = train_test_split( X, y ,test_size=0.1, random_state=0) 

# kaze applied

In [ ]:
kaze_train = get_kaze(train_imgs, name='kaze_train')
kaze_val = get_kaze(val_imgs, name='kaze_val')
kaze_test = get_kaze(X_test, name='kaze_test')

In [78]:
nsamples, nx, ny 

In [77]:
nsamples

(16145, 64, 64, 3)

In [82]:
kaze_val.shape, kaze_test.shape

((1794, 2048), (4485, 2048))

In [67]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
X_train = sc.fit_transform(train_imgs)
X_test = sc.transform(X_test)

ValueError: Found array with dim 4. MinMaxScaler expected <= 2.

In [59]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [60]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
lda = LDA(n_components = 2)
X_train = lda.fit_transform(X_train, y_train)
X_test = lda.transform(X_test)